In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')

Connecting celiib@10.28.0.34:3306


In [3]:
#ta3p100.ProofreadLabelOrphan() & "segment_id=648518346341352006"

In [4]:
#ta3p100.CleansedMeshOrphan()

In [5]:
@schema
class CoarseLabelOrphan(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabelOrphan
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabelOrphan
    """
    
    key_source = ta3p100.ProofreadLabelOrphan & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabelOrphan & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMeshOrphan & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [6]:
start = time.time()
CoarseLabelOrphan.populate(reserve_jobs=True)
print(time.time() - start)

648518346341396164
Segment 648518346341396164 vertex labels regenerated in: 1.0520217418670654 seconds.
648518346341396536
Segment 648518346341396536 vertex labels regenerated in: 0.9654598236083984 seconds.
648518346341397739
Segment 648518346341397739 vertex labels regenerated in: 1.4176890850067139 seconds.
648518346341398503
Segment 648518346341398503 vertex labels regenerated in: 1.1641592979431152 seconds.
648518346341399051
Segment 648518346341399051 vertex labels regenerated in: 1.0636241436004639 seconds.
648518346341399296
Segment 648518346341399296 vertex labels regenerated in: 1.1531801223754883 seconds.
648518346341399962
Segment 648518346341399962 vertex labels regenerated in: 1.0453083515167236 seconds.
648518346341400285
Segment 648518346341400285 vertex labels regenerated in: 0.9692349433898926 seconds.
648518346341401013
Segment 648518346341401013 vertex labels regenerated in: 0.865281343460083 seconds.
648518346341401776
Segment 648518346341401776 vertex labels regen

In [7]:
#(schema.jobs & "table_name='__coarse_label_orphan'")#.delete()

In [8]:
#ta3p100.CoarseLabelOrphan()